In [ ]:
# --- ENTERPRISE GRADE FACE SWAP SERVER (v2.1 - NO SCRAMBLE FIX) ---
# Engineered by R H A Ashan Imalka ( SCXR )
# Focus: Stability, Error-Correction, High Performance, ZERO ARTIFACTS

import os
import sys
import site
import shutil
import subprocess
from google.colab import drive

# ==========================================
# PHASE 1: ENVIRONMENT PREPARATION
# ==========================================

print(" [SYSTEM] Initializing Enterprise Environment...")
print(" [INSTALL] Installing High-Performance Dependencies...")

!pip install insightface==0.7.3 onnxruntime-gpu flask flask-cors pyngrok gfpgan basicsr realesrgan > /dev/null 2>&1

# ==========================================
# PHASE 2: SURGICAL PATCHING (THE FIX)
# ==========================================
def patch_broken_dependency():
    print(" [PATCH] Scanning system for broken 'basicsr' module...")
    possible_paths = site.getsitepackages()
    if hasattr(site, 'getusersitepackages'):
        possible_paths.append(site.getusersitepackages())

    patch_count = 0
    for package_dir in possible_paths:
        target_file = os.path.join(package_dir, 'basicsr', 'data', 'degradations.py')
        if os.path.exists(target_file):
            try:
                with open(target_file, 'r') as f:
                    code = f.read()
                bad_import = 'from torchvision.transforms.functional_tensor import rgb_to_grayscale'
                good_import = 'from torchvision.transforms.functional import rgb_to_grayscale'
                if bad_import in code:
                    print(" [PATCH] Applying surgical fix...")
                    new_code = code.replace(bad_import, good_import)
                    with open(target_file, 'w') as f:
                        f.write(new_code)
                    patch_count += 1
                    print(" [SUCCESS] Module patched.")
            except Exception as e:
                print(f" [ERROR] Patch failed: {e}")

    if patch_count == 0:
        print(" [INFO] System appears clean or already patched.")

patch_broken_dependency()

# ==========================================
# PHASE 3: MODEL AQUISITION
# ==========================================
def download_model(url, filename, description):
    if not os.path.exists(filename):
        print(f" [DOWNLOAD] Retrieving {description}...")
        subprocess.run(['wget', '-q', url, '-O', filename])
    else:
        print(f" [CACHE] {description} found locally.")

download_model('https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx', 'inswapper_128.onnx', 'InSwapper Model')
download_model('https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.4.pth', 'GFPGANv1.4.pth', 'GFPGAN Face Enhancer')
download_model('https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth', 'RealESRGAN_x4plus.pth', 'RealESRGAN Upscaler')

# ==========================================
# PHASE 4: CORE INITIALIZATION
# ==========================================
print(" [BOOT] Starting AI Engines...")

import cv2
import insightface
import numpy as np
import base64
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from gfpgan import GFPGANer
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer

# CONFIGURATION
NGROK_AUTH_TOKEN = "35bsndR53PzXmMZoQYQ60IVLB0O_51qEwEWaXvsMvnukGfaeK"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

app = Flask(__name__)
CORS(app)

# 1. Detection
app_face = insightface.app.FaceAnalysis(name='buffalo_l')
app_face.prepare(ctx_id=0, det_size=(640, 640))

# 2. Swapping
swapper = insightface.model_zoo.get_model('inswapper_128.onnx', download=False, download_zip=False)

# 3. Restoration (GFPGAN)
face_enhancer = GFPGANer(model_path='GFPGANv1.4.pth', upscale=1, arch='clean', channel_multiplier=2, bg_upsampler=None)

# 4. Upscaling (RealESRGAN)
print(" [CONFIG] Configuring Upscaler for Maximum Coherence...")
model_rrdb = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
bg_upsampler = RealESRGANer(
    scale=2,
    model_path='RealESRGAN_x4plus.pth',
    model=model_rrdb,
    tile=0,          # <--- CHANGED TO 0! Disables tiling logic completely. No more scrambled eggs!
    tile_pad=10,     # Ignored when tile=0, but kept for safety
    pre_pad=0,
    half=True
)

print(" [READY] All AI Systems Operational.")

# ==========================================
# PHASE 5: LOGIC
# ==========================================
def decode_image(base64_string):
    if ',' in base64_string:
        base64_string = base64_string.split(',')[1]
    img_data = base64.b64decode(base64_string)
    nparr = np.frombuffer(img_data, np.uint8)
    return cv2.imdecode(nparr, cv2.IMREAD_COLOR)

def encode_image(cv2_img):
    _, buffer = cv2.imencode('.jpg', cv2_img)
    return "data:image/jpeg;base64," + base64.b64encode(buffer).decode('utf-8')

def get_largest_face(faces):
    if not faces:
        return None
    return max(faces, key=lambda x: (x.bbox[2] - x.bbox[0]) * (x.bbox[3] - x.bbox[1]))

@app.route('/')
def home():
    return "Asagi Enterprise Server: ONLINE (No-Scramble Mode)"

@app.route('/swap', methods=['POST'])
def swap_faces():
    try:
        data = request.json
        source_b64 = data.get('source')
        target_b64 = data.get('target')
        do_face_enhance = data.get('face_enhance', False)
        do_full_upscale = data.get('full_upscale', False)

        if not source_b64 or not target_b64:
            return jsonify({'error': 'Missing input payload'}), 400

        source_img = decode_image(source_b64)
        target_img = decode_image(target_b64)

        # Detection
        source_faces = app_face.get(source_img)
        target_faces = app_face.get(target_img)

        if not source_faces:
            return jsonify({'error': 'Source: No face detected.'}), 400
        if not target_faces:
            return jsonify({'error': 'Target: No face detected.'}), 400

        source_face = get_largest_face(source_faces)

        # Processing
        res_img = target_img.copy()

        # Swap
        for target_face in target_faces:
            res_img = swapper.get(res_img, target_face, source_face, paste_back=True)

        # Enhance
        if do_face_enhance or do_full_upscale:
            try:
                _, _, res_img = face_enhancer.enhance(res_img, has_aligned=False, only_center_face=False, paste_back=True)
            except Exception as e:
                print(f" [WARN] Face Enhance failed: {e}")

        # Upscale
        if do_full_upscale:
            try:
                # With tile=0, this might use a lot of VRAM, but result will be perfect
                output, _ = bg_upsampler.enhance(res_img, outscale=2)
                res_img = output
            except RuntimeError as e:
                if "out of memory" in str(e):
                    print(" [WARN] Image too big for Upscale! Returning normal resolution.")
                    return jsonify({'error': 'Image too big for Ultra Upscale. Try turning off Ultra Upscale.'}), 500
                else:
                    print(f" [ERROR] Upscale failed: {e}")

        result_b64 = encode_image(res_img)
        return jsonify({'result': result_b64, 'status': 'success'})

    except Exception as e:
        print(f" [ERROR] Processing failed: {e}")
        return jsonify({'error': str(e)}), 500

# ==========================================
# PHASE 6: LAUNCH
# ==========================================
ngrok.kill()
try:
    tunnel = ngrok.connect(5000)
    public_url = tunnel.public_url
    print(f"\n\n [NETWORK] TUNNEL ESTABLISHED")
    print(f" [LINK] >>>> {public_url} <<<<")
    app.run(port=5000)
except Exception as e:
    print(f" [CRITICAL] Network Failure: {e}")

 [SYSTEM] Initializing Enterprise Environment...
 [INSTALL] Installing High-Performance Dependencies...
 [PATCH] Scanning system for broken 'basicsr' module...
 [INFO] System appears clean or already patched.
 [CACHE] InSwapper Model found locally.
 [CACHE] GFPGAN Face Enhancer found locally.
 [CACHE] RealESRGAN Upscaler found locally.
 [BOOT] Starting AI Engines...
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'fuse_conv_bias': '0', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_ext

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 12:27:29] "OPTIONS / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 12:27:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 12:28:03] "OPTIONS /swap HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Nov/2025 12:28:13] "POST /swap HTTP/1.1" 200 -
